# Quasi-cord の接続行列

## 準備

In [1]:
import numpy as np

In [2]:
Labels = ['T','L','B','R']

class segment:
    def __init__(self, ends):
        self.ends = ends
        
    def show(self):
        return self.ends

## Quasi-cord 定義

In [3]:
c = [3,2,1,2] #[3,2,1,4] #[3,4,5,2] #[1,3,1,1] #
s = sum(c) + 1

## end points

In [4]:
endpoints = []

for i in range(4):
    points = [Labels[i]+str(k) for k in range(c[i])]
    points += list(reversed(points))
    if i%2==1:
        idx = int(len(points)/2)
        points.insert(idx, Labels[i])
    endpoints.append(points)
    
print(endpoints)

[['T0', 'T1', 'T2', 'T2', 'T1', 'T0'], ['L0', 'L1', 'L', 'L1', 'L0'], ['B0', 'B0'], ['R0', 'R1', 'R', 'R1', 'R0']]


## segments

In [5]:
all_segs = []

topleft = (c[0]+c[1])-(c[2]+c[3])
for i in range(topleft):
    seg = segment([endpoints[0][i], endpoints[1][i]])
    all_segs.append(seg)

remaining = []
remaining.append(list(reversed(endpoints[0][topleft:]))+endpoints[1][topleft:])
remaining.append(endpoints[3]+endpoints[2])
#print(remaining)

for i in range(s-topleft):
    all_segs.append(segment([remaining[0][i], remaining[1][i]]))

print([s.show() for s in all_segs])

[['T0', 'L0'], ['T1', 'L1'], ['T0', 'R0'], ['T1', 'R1'], ['T2', 'R'], ['T2', 'R1'], ['L', 'R0'], ['L1', 'B0'], ['L0', 'B0']]


## 接続行列の生成

In [6]:
M = np.zeros((s,s+1), dtype=int)

columns = sum([endpoints[i][:c[i]+(i%2)] for i in range(4)],[])
#print(columns)

for i in range(len(all_segs)):
    seg = all_segs[i]
#    print(seg.show(), columns.index(seg.ends[0]), columns.index(seg.ends[1]))
    M[i][columns.index(seg.ends[0])] = 1
    M[i][columns.index(seg.ends[1])] = -1

C = np.roll(np.eye(1, s+1, dtype=int), shift=c[0]+c[1], axis=1)
M = np.r_[M, C]
print(M)

[[ 1  0  0 -1  0  0  0  0  0  0]
 [ 0  1  0  0 -1  0  0  0  0  0]
 [ 1  0  0  0  0  0  0 -1  0  0]
 [ 0  1  0  0  0  0  0  0 -1  0]
 [ 0  0  1  0  0  0  0  0  0 -1]
 [ 0  0  1  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  1  0 -1  0  0]
 [ 0  0  0  0  1  0 -1  0  0  0]
 [ 0  0  0  1  0  0 -1  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0]]


In [7]:
np.linalg.det(M)

-1.0

## Experiments

In [13]:
def elem_matrix(n,i,j):
    P = np.eye(n,n, dtype=int)
    P[i][j] = 1
    return P
def P(n,i,j):
    return elem_matrix(n,i,j)

In [22]:
N = M
N = N@P(s+1,1,0)
N = P(s+1,0,1)@N

print(N)

[[ 2  1  0 -1 -1  0  0  0  0  0]
 [ 1  1  0  0 -1  0  0  0  0  0]
 [ 1  0  0  0  0  0  0 -1  0  0]
 [ 1  1  0  0  0  0  0  0 -1  0]
 [ 0  0  1  0  0  0  0  0  0 -1]
 [ 0  0  1  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  1  0 -1  0  0]
 [ 0  0  0  0  1  0 -1  0  0  0]
 [ 0  0  0  1  0  0 -1  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0]]


In [10]:
print(c)

[3, 2, 1, 2]


## TODO

[x] 接続行列の生成について。quasi-cord が $\tau + \lambda = \beta + \rho$ の場合しかうまく行ってない。